## Plot and view contact maps and ratios from a sims config file and pre-computed .npy contact maps

In [ ]:
import os
import sys
import yaml

import numpy as np
from matplotlib.colors import LogNorm
from matplotlib.ticker import EngFormatter
import matplotlib.pyplot as plt
import seaborn as sns
import cooltools.lib.plotting

import ipywidgets as widgets
from ipywidgets import Layout

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
CONFIG_FILE = '/users/emma.rusch/goloborodko-lab/code/baxterLab2020/src/sims/yeast_cen_intertwines_simple/configs/yeast_cen_intertwines_sweep.config.yml'
OUTPUT = widgets.Output()

    config = yaml.load(yml_file, Loader=yaml.FullLoader)

results_dir = config['maps_dir']
files = os.listdir(results_dir)

In [ ]:
def plot_map(hmaps, titles, ratio):
    with OUTPUT:
        f, ax = plt.subplots(
            figsize=(20, 18),
            ncols=1 if len(hmaps) == 1 or ratio is True else 2)
        bp_formatter = EngFormatter('b')

        if len(hmaps) > 1:
            if ratio:
                # plot ratio map
                hmap = hmaps[0] / hmaps[1]
                img = ax.matshow(np.log10(hmap), vmin=-1, vmax=1, cmap='RdBu')
                ax.set_title(f'{titles[0]} / {titles[1]}')
            else:
                ax[0].matshow(hmaps[0], norm=LogNorm(vmin=1, vmax=hmaps[0].max()), cmap='fall')
                img = ax[1].matshow(hmaps[1], norm=LogNorm(vmin=1, vmax=hmaps[1].max()), cmap='fall')
                ax[0].set_title(titles[0])
                ax[1].set_title(titles[1])
        else:
            img = ax.matshow(hmaps[0], norm=LogNorm(vmin=1, vmax=hmaps[0].max()), cmap='fall')
            ax.set_title(titles[0])

        plt.colorbar(img, ax=ax ,fraction=0.046, pad=0.04, label='counts (log)')
        
        plt.show()

In [ ]:
def callback_plot(change):
    files = files_list.value
    ratio = plot_ratio.value
    with OUTPUT:
        OUTPUT.clear_output()
        hmaps = [np.load(os.path.join(results_dir, f + '.npy')) for f in files]
        plot_map(hmaps, files, ratio)

In [ ]:
display(widgets.Label('Select up to 2:'))
files_list = widgets.SelectMultiple(
    options=[os.path.splitext(f)[0] for f in sorted(files)],
    description='',
    style={'description_width': 'initial'},
    layout=Layout(height='100%', width='initial'),
    disabled=False
)

files_list.observe(callback_plot, names='value')

plot_ratio = widgets.Checkbox(
    value=False,
    description='Ratio'
)

plot_ratio.observe(callback_plot, names='value')

# TODO: implement filters
param_label = widgets.Label('Filter by parameters:')
param_widgets = {key: widgets.RadioButtons(
        options=value,
        value=None,
        description=key[2:]
    ) for key, value in config['cli_params'].items()}
param_clear_btn = widgets.Button(
    description='Clear filter'
)

def clear_filter(change):
    for widget in param_widgets.values():
        widget.value = None

param_clear_btn.on_click(clear_filter)

param_list = widgets.VBox([param_label, *param_widgets.values(), param_clear_btn])

nav = widgets.VBox([plot_ratio, files_list])

main = widgets.HBox([nav, OUTPUT])

display(main)